# Chase Credit Card Spending Analysis

Drop your Chase CSV export into `../data/chase_export.csv` and run all cells.

**Sections:**
1. Setup & Data Loading
2. Summary Stats
3. Monthly Spend
4. Spend by Category
5. Top Merchants
6. Subscription Detection

---
## 1. Setup & Data Loading

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import seaborn as sns
from pathlib import Path
import warnings

warnings.filterwarnings('ignore')
sns.set_theme(style='whitegrid', palette='muted')
plt.rcParams['figure.figsize'] = (12, 5)
plt.rcParams['axes.titlesize'] = 14

DATA_PATH = Path('../data/chase_export.csv')

if not DATA_PATH.exists():
    raise FileNotFoundError(
        f"Chase CSV not found at {DATA_PATH.resolve()}\n"
        "Export your transactions from chase.com and save the file there."
    )

raw = pd.read_csv(DATA_PATH)
print(f"Loaded {len(raw):,} rows")
raw.head()

In [ ]:
# Normalize column names (strip whitespace, title-case)
raw.columns = raw.columns.str.strip()

df = raw.copy()

# Parse dates
df['Transaction Date'] = pd.to_datetime(df['Transaction Date'])
df['Post Date'] = pd.to_datetime(df['Post Date'])

# Chase amounts: negative = expense, positive = payment/credit
# Keep only debits (Sales / expenses)
df = df[df['Amount'] < 0].copy()

# Make Amount positive for readability
df['Amount'] = df['Amount'].abs()

# Derived time columns
df['Year']      = df['Transaction Date'].dt.year
df['Month']     = df['Transaction Date'].dt.month
df['YearMonth'] = df['Transaction Date'].dt.to_period('M')
df['Week']      = df['Transaction Date'].dt.to_period('W')

# Clean merchant name (trim whitespace)
df['Description'] = df['Description'].str.strip()

print(f"Expenses only: {len(df):,} transactions")
print(f"Date range: {df['Transaction Date'].min().date()} → {df['Transaction Date'].max().date()}")
df.dtypes

---
## 2. Summary Stats

In [ ]:
total_spend      = df['Amount'].sum()
n_months         = df['YearMonth'].nunique()
avg_monthly      = total_spend / n_months if n_months else 0
monthly_totals   = df.groupby('YearMonth')['Amount'].sum()
biggest_month    = monthly_totals.idxmax()
biggest_month_amt = monthly_totals.max()
biggest_category = df.groupby('Category')['Amount'].sum().idxmax()
biggest_cat_amt  = df.groupby('Category')['Amount'].sum().max()
n_merchants      = df['Description'].nunique()

summary = {
    'Total Spend':           f'${total_spend:,.2f}',
    'Months of Data':         str(n_months),
    'Avg Monthly Spend':     f'${avg_monthly:,.2f}',
    'Biggest Month':         f'{biggest_month} (${biggest_month_amt:,.2f})',
    'Biggest Category':      f'{biggest_category} (${biggest_cat_amt:,.2f})',
    'Unique Merchants':       str(n_merchants),
    'Total Transactions':    f'{len(df):,}',
}

summary_df = pd.DataFrame.from_dict(summary, orient='index', columns=['Value'])
summary_df.index.name = 'Metric'
summary_df

---
## 3. Monthly Spend

In [ ]:
monthly = (
    df.groupby('YearMonth')['Amount']
    .sum()
    .reset_index()
    .rename(columns={'Amount': 'Total'})
    .sort_values('YearMonth')
)
monthly['YearMonth_str'] = monthly['YearMonth'].astype(str)
monthly['Cumulative']    = monthly['Total'].cumsum()
monthly['MoM_Delta']     = monthly['Total'].diff()

# Bar chart — monthly spend
fig, ax = plt.subplots()
bars = ax.bar(monthly['YearMonth_str'], monthly['Total'], color=sns.color_palette('muted')[0])
ax.yaxis.set_major_formatter(mticker.FuncFormatter(lambda x, _: f'${x:,.0f}'))
ax.set_xlabel('Month')
ax.set_ylabel('Spend ($)')
ax.set_title('Monthly Spend')
plt.xticks(rotation=45, ha='right')
# Add value labels
for bar in bars:
    h = bar.get_height()
    ax.text(bar.get_x() + bar.get_width() / 2, h + 10, f'${h:,.0f}',
            ha='center', va='bottom', fontsize=8)
plt.tight_layout()
plt.show()

In [ ]:
# Cumulative spend line
fig, ax = plt.subplots()
ax.plot(monthly['YearMonth_str'], monthly['Cumulative'], marker='o', linewidth=2)
ax.fill_between(monthly['YearMonth_str'], monthly['Cumulative'], alpha=0.15)
ax.yaxis.set_major_formatter(mticker.FuncFormatter(lambda x, _: f'${x:,.0f}'))
ax.set_xlabel('Month')
ax.set_ylabel('Cumulative Spend ($)')
ax.set_title('Cumulative Spend Over Time')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

In [ ]:
# Month-over-month delta
delta = monthly.dropna(subset=['MoM_Delta'])
colors = ['#d62728' if x > 0 else '#2ca02c' for x in delta['MoM_Delta']]
fig, ax = plt.subplots()
ax.bar(delta['YearMonth_str'], delta['MoM_Delta'], color=colors)
ax.axhline(0, color='black', linewidth=0.8)
ax.yaxis.set_major_formatter(mticker.FuncFormatter(lambda x, _: f'${x:,.0f}'))
ax.set_xlabel('Month')
ax.set_ylabel('Change vs Prior Month ($)')
ax.set_title('Month-over-Month Spend Change (red = more, green = less)')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

monthly[['YearMonth_str', 'Total', 'Cumulative', 'MoM_Delta']].rename(
    columns={'YearMonth_str': 'Month', 'Total': 'Spend', 'MoM_Delta': 'vs Prior Month'}
).style.format({'Spend': '${:,.2f}', 'Cumulative': '${:,.2f}', 'vs Prior Month': '${:,.2f}'})

---
## 4. Spend by Category

In [ ]:
cat = (
    df.groupby('Category')['Amount']
    .agg(['sum', 'count'])
    .rename(columns={'sum': 'Total', 'count': 'Transactions'})
    .sort_values('Total', ascending=False)
    .reset_index()
)
cat['% of Spend'] = (cat['Total'] / cat['Total'].sum() * 100).round(1)

fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Bar chart
axes[0].barh(cat['Category'][::-1], cat['Total'][::-1],
             color=sns.color_palette('muted', len(cat)))
axes[0].xaxis.set_major_formatter(mticker.FuncFormatter(lambda x, _: f'${x:,.0f}'))
axes[0].set_xlabel('Total Spend ($)')
axes[0].set_title('Spend by Category')

# Pie chart (top 8 + Other)
TOP_N = 8
top = cat.head(TOP_N).copy()
other_total = cat.iloc[TOP_N:]['Total'].sum()
if other_total > 0:
    other_row = pd.DataFrame([{'Category': 'Other', 'Total': other_total}])
    top = pd.concat([top, other_row], ignore_index=True)
axes[1].pie(
    top['Total'],
    labels=top['Category'],
    autopct='%1.1f%%',
    startangle=140,
    colors=sns.color_palette('muted', len(top))
)
axes[1].set_title('Category Share of Total Spend')

plt.tight_layout()
plt.show()

cat.style.format({'Total': '${:,.2f}', '% of Spend': '{:.1f}%'})

In [ ]:
# Category spend over time (stacked area)
top_cats = cat['Category'].head(6).tolist()
cat_time = (
    df[df['Category'].isin(top_cats)]
    .groupby(['YearMonth', 'Category'])['Amount']
    .sum()
    .unstack(fill_value=0)
    .sort_index()
)
cat_time.index = cat_time.index.astype(str)

fig, ax = plt.subplots(figsize=(14, 6))
cat_time.plot.area(ax=ax, alpha=0.75, colormap='tab10')
ax.yaxis.set_major_formatter(mticker.FuncFormatter(lambda x, _: f'${x:,.0f}'))
ax.set_xlabel('Month')
ax.set_ylabel('Spend ($)')
ax.set_title('Top Category Spend Over Time')
ax.legend(loc='upper left', bbox_to_anchor=(1, 1))
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

---
## 5. Top Merchants

In [ ]:
TOP_MERCHANTS = 20

merchants = (
    df.groupby('Description')['Amount']
    .agg(['sum', 'count', 'mean'])
    .rename(columns={'sum': 'Total', 'count': 'Visits', 'mean': 'Avg per Visit'})
    .sort_values('Total', ascending=False)
    .head(TOP_MERCHANTS)
    .reset_index()
)

fig, ax = plt.subplots(figsize=(12, 8))
palette = sns.color_palette('muted', len(merchants))
ax.barh(merchants['Description'][::-1], merchants['Total'][::-1], color=palette)
ax.xaxis.set_major_formatter(mticker.FuncFormatter(lambda x, _: f'${x:,.0f}'))
ax.set_xlabel('Total Spend ($)')
ax.set_title(f'Top {TOP_MERCHANTS} Merchants by Total Spend')
for i, (total, visits) in enumerate(zip(merchants['Total'][::-1], merchants['Visits'][::-1])):
    ax.text(total + 5, i, f'  {visits}x', va='center', fontsize=8, color='gray')
plt.tight_layout()
plt.show()

merchants.style.format({
    'Total': '${:,.2f}',
    'Avg per Visit': '${:,.2f}'
})

---
## 6. Subscription Detection

Heuristic: flag merchants that charge on a **regular cadence** (weekly ±2 days, monthly ±5 days)
with **consistent amounts** (std dev < 15% of mean).

In [ ]:
def detect_subscriptions(df, min_occurrences=2):
    """Return a DataFrame of likely subscription charges."""
    results = []

    for merchant, group in df.groupby('Description'):
        if len(group) < min_occurrences:
            continue

        dates  = group['Transaction Date'].sort_values().reset_index(drop=True)
        amounts = group['Amount']

        # Gaps between consecutive charges (days)
        gaps = dates.diff().dropna().dt.days

        if len(gaps) == 0:
            continue

        avg_gap = gaps.mean()
        std_gap = gaps.std() if len(gaps) > 1 else 0

        # Classify cadence
        if 5 <= avg_gap <= 9 and std_gap <= 2:
            cadence = 'Weekly'
            monthly_est = amounts.mean() * 4.33
        elif 25 <= avg_gap <= 35 and std_gap <= 5:
            cadence = 'Monthly'
            monthly_est = amounts.mean()
        elif 85 <= avg_gap <= 95 and std_gap <= 7:
            cadence = 'Quarterly'
            monthly_est = amounts.mean() / 3
        elif 355 <= avg_gap <= 375 and std_gap <= 10:
            cadence = 'Annual'
            monthly_est = amounts.mean() / 12
        else:
            continue  # no clear cadence

        # Amount consistency check: CV < 15%
        cv = (amounts.std() / amounts.mean()) if amounts.mean() > 0 else 1
        if cv > 0.15:
            continue

        results.append({
            'Merchant':          merchant,
            'Cadence':           cadence,
            'Occurrences':       len(group),
            'Avg Charge':        amounts.mean(),
            'Est Monthly Cost':  monthly_est,
            'First Seen':        dates.iloc[0].date(),
            'Last Seen':         dates.iloc[-1].date(),
        })

    return (
        pd.DataFrame(results)
        .sort_values('Est Monthly Cost', ascending=False)
        .reset_index(drop=True)
    )


subs = detect_subscriptions(df)
print(f"Detected {len(subs)} likely subscriptions")
subs

In [ ]:
if len(subs) > 0:
    total_sub_monthly = subs['Est Monthly Cost'].sum()
    print(f"Estimated monthly subscription total: ${total_sub_monthly:,.2f}")
    print(f"Estimated annual subscription total:  ${total_sub_monthly * 12:,.2f}")

    fig, ax = plt.subplots(figsize=(10, max(4, len(subs) * 0.4 + 1)))
    ax.barh(subs['Merchant'][::-1], subs['Est Monthly Cost'][::-1],
            color=sns.color_palette('muted', len(subs)))
    ax.xaxis.set_major_formatter(mticker.FuncFormatter(lambda x, _: f'${x:,.2f}'))
    ax.set_xlabel('Est. Monthly Cost ($)')
    ax.set_title('Detected Subscriptions — Estimated Monthly Cost')
    for i, (cost, cadence) in enumerate(
        zip(subs['Est Monthly Cost'][::-1], subs['Cadence'][::-1])
    ):
        ax.text(cost + 0.3, i, f'  {cadence}', va='center', fontsize=8, color='gray')
    plt.tight_layout()
    plt.show()

    subs.style.format({
        'Avg Charge': '${:,.2f}',
        'Est Monthly Cost': '${:,.2f}'
    })
else:
    print("No subscriptions detected. Try lowering min_occurrences or relaxing cadence tolerances.")